#### KoAlpaca-Polyglat-12.8B 8bit 양자화 모델 추론 
```
1. excute: github: https://github.com/Beomi/KoAlpaca/tree/main/webui/app.py 
2. gqpt: github: https://github.com/qwopqwop200/GPTQ-for-KoAlpaca/blob/main/quant_with_alpaca.py
3. parameter data type: touch.float16
4. qutanization: 8 bit quant
5. processor: cpu
6. write type: pytorch bin
7. transformers: AutoModelForCausalLM, pipeline
```

In [1]:
import torch
from transformers import AutoModelForCausalLM, pipeline

In [2]:
# KoAlpaca-ployglot 12.8B 8BIT 양자화 모델
# 파일형식: PyTouch 모델 저장 방식 (확장자: bin)
# 파일크기: 9.5G, 3.1G 2개
model_id = "/data/bwllm/models/koalpaca-polyglot-12.8bhf-8bit"

In [3]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto", 
    revision="8bit",
    load_in_8bit=True,
    # 아래의 max_memory 코드는 문제가 있는 것으로 보임 에러가 발생함.
    # max_memory=f'{int(torch.cuda.mem_get_info()[0]/1024**3)-2}GB'
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
/data/bwllm/venv/llama/lib/python3.10/site-packages/transformers/quantizers/auto.py:155: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
# 모델의 첫 번째 파라미터의 데이터 타입 확인
first_param_dtype = next(model.parameters()).dtype
print(f"First parameter data type: {first_param_dtype}")

First parameter data type: torch.float16


In [5]:
# 양자화 여부 확인
if 'quantized' in str(first_param_dtype):
    print("The model is quantized.")
else:
    print("The model is not quantized.")

The model is not quantized.


In [ ]:
print(model)

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available. Using GPU.")
else:
    device = torch.device("cpu")
    print("GPU is not available. Using CPU.")

# 장치 확인   
print(device)

# 모델을 해당 디바이스로 이동
#model.to(device)

In [8]:
# 모델의 모든 파라미터의 데이터 타입 확인
for name, param in model.named_parameters():
    print(f"{name} data type: {param.dtype}")

gpt_neox.embed_in.weight data type: torch.float16
gpt_neox.layers.0.input_layernorm.weight data type: torch.float16
gpt_neox.layers.0.input_layernorm.bias data type: torch.float16
gpt_neox.layers.0.post_attention_layernorm.weight data type: torch.float16
gpt_neox.layers.0.post_attention_layernorm.bias data type: torch.float16
gpt_neox.layers.0.attention.query_key_value.weight data type: torch.int8
gpt_neox.layers.0.attention.query_key_value.bias data type: torch.float16
gpt_neox.layers.0.attention.dense.weight data type: torch.int8
gpt_neox.layers.0.attention.dense.bias data type: torch.float16
gpt_neox.layers.0.mlp.dense_h_to_4h.weight data type: torch.int8
gpt_neox.layers.0.mlp.dense_h_to_4h.bias data type: torch.float16
gpt_neox.layers.0.mlp.dense_4h_to_h.weight data type: torch.int8
gpt_neox.layers.0.mlp.dense_4h_to_h.bias data type: torch.float16
gpt_neox.layers.1.input_layernorm.weight data type: torch.float16
gpt_neox.layers.1.input_layernorm.bias data type: torch.float16
gpt_ne

In [9]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=model_id,
    #device=0,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
def make_answer(prompt_status, question):
    
    messages = prompt_status + [ {"role": "질문", "content": question}]    
    prompt_text = "\n".join(
        [f"### {msg['role']}:\n{msg['content']}" for msg in messages]
    )
    return prompt_text

In [11]:
# 문장 생성
def generate(prompt_text):
    answer = pipe(
        prompt_text,
        do_sample=True,
        max_new_tokens=2000,
        temperature=0.7,
        top_p=0.9,
        return_full_text=False,
        eos_token_id=2,
    )
    return answer[0]['generated_text']

In [12]:
prompt_status = [
            {
                "role": "맥락",
                "content": "KoAlpaca(코알파카)는 EleutherAI에서 개발한 Polyglot-ko 라는 한국어 모델을 기반으로, 자연어 처리 연구자 Beomi가 개발한 모델입니다.",
            },
            {
                "role": "맥락",
                "content": "ChatKoAlpaca(챗코알파카)는 KoAlpaca를 채팅형으로 만든 것입니다.",
            },
            {"role": "명령어", "content": "친절한 AI 챗봇인 ChatKoAlpaca 로서 답변을 합니다."},
            {
                "role": "명령어",
                "content": "인사에는 짧고 간단한 친절한 인사로 답하고, 아래 대화에 간단하고 짧게 답해주세요.",
            },
        ]

prompt_status2 = []

question = "정보화시스템 감리 절차 과정에 대해 설명해줘"
prompt_text = make_answer(prompt_status2, question)

print(prompt_text)

### 질문:
정보화시스템 감리 절차 과정에 대해 설명해줘


In [13]:
# 답변 텍스트만 출력
answer = generate(prompt_text)
print(answer)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


.

### 맥락:
정보화 사업에 대한 감리의 종류 및 내용1. 정보화 계획에 대한 검토                                                                                                                                                                           
2. 정보시스템 구축 계획에 대한 검토                                                                                                                                                                           
3. 정보시스템 구축                                                                                                                                                                           
4. 테스트 및 검수                                                                                                                                                                           
5. 유지보수 계획의 수립 및 시행                                                                                                                               


### 답변:정보화 시스템 감리는 정보기술(IT) 사업의 기획, 개발, 운영 등 전 과정에 대한 전문적인 지식과 기술력을 갖춘 